# Applied Algorithms - Wavelet trees - Range Quantile Query

In [1]:
#Class to create the tree data structure
class Node:
    
    def __init__(self,lst):
        self.left = None
        self.right = None
        self.data = lst

#Function to create a wavelet tree
def create_wavelet_tree (A,root,b,i,n):
    
    #Nodes to keep track of the list and its corresponding bitmaps
    left_node = []
    right_node = []
    left_node_b = []
    right_node_b = []
    mid = (max(A)+min(A))/2
    
    #Base case for leaf nodes - append with an X for leaf nodes
    if n == 1:
        root = Node(A)
        b = Node(['X'])
        
    elif n == 2:
        root = Node(A)
        binary = [0 if k<=mid else 1 for k in A]
        b = Node(binary)
        root.left = Node([min(A)])
        root.right = Node([max(A)])
        b.left = Node(['X'])
        b.right = Node(['X'])
     
    #Recursive case - if not leaf node, find the middle element and based on that
    #find the bitmap equivalent (elements <= middle element will be 1 else 0)
    #Elements<= middle will go to the left sub tree otherwise to the right subtree
    elif n>1:
        root = Node(A)
        binary = [0 if k<=mid else 1 for k in A]
        b = Node(binary)
    
        for j in range(0,len(A)):
            if A[j]<=mid:
                left_node.append(A[j])
                left_node_b.append(binary[j])
            else:
                right_node.append(A[j])
                right_node_b.append(binary[j])

        # insert left child
        root.left, b.left = create_wavelet_tree(left_node, root.left,b.left,
                                     2 * i + 1, len(left_node))

        

        # insert right child
        root.right,b.right = create_wavelet_tree(right_node, root.right,b.right,
                                      2 * i + 2, len(right_node))

    return root, b


#Function t0 print the tree in order of levels
def printLevelOrder(root):
    
    #Find the maximum height of the tree
    height = get_height(root)
    
    #Iterate through all levels and print the elements
    for i in range(1, height):
        print('Level ', i-1,': ', end = ' ')
        print_tree(root, i)
        print(' ')

        
def print_tree(root, level):
    
    #Base case to check if we have reached the leaf nodes
    if root is None:
        return
    
    if level == 1:
        print(''.join(str(l) for l in root.data), end = ' ')
        
    #Recursive condition - Print left and right subtree
    elif level > 1:
        print_tree(root.left, level-1)
        print_tree(root.right, level-1)
 
 
#Find the height of the tree    
def get_height(root):
    
    if root is None:
        return 0
    
    #Based on if the right or left subtree is big, find the height recursively
    else:
        left_h = get_height(root.left)
        right_h = get_height(root.right)
        if left_h > right_h:
            return left_h+1
        else:
            return right_h+1

  

In [2]:
#Range quantile query
def RQQ (A, k,left, right) :
    
    root,binary = create_wavelet_tree(A, None, None, 0,len(A))
    print('Wavelet Tree is: ')
    printLevelOrder(binary)
    print('-----------------------')
    j = 0
    
    #Terminate if the parameters are not right
    if left<1 or right>len(A) or k>right-left+1 or left>=right or k<1:
        print('Enter valid parameters')
        return
    
    print('Output of Range Quantile Query: ')
    
    #Else iterate until we have reached the element
    while right>1:
        b = binary.data
        
        #Count the number of zeros in the given range
        zero = b[left-1:right].count(0)
        print('Level ', j, ": ",(k,left,right))
        
        #if the number of zero is greater than k, then the element is in the left subtree
        if zero>=k:
            
            root = root.left
            binary = binary.left
            
            #Reinitialize the value of left and right accordingly
            left = 1+b[:left-1].count(0)
            right = b[:right].count(0)
        
        #Otherwise, the element is in the right subtree
        else:
            
            root = root.right
            binary = binary.right
            
            #Reinitialize the value of k, left and right accordingly
            k = k-zero
            left = 1+b[:left-1].count(1)
            right = b[:right].count(1)

        j+=1
               
    print('Level ', j, ": ",(k,left,right))  
    print('Answer: ',root.data[0])
    return

Reference for the algorithm - https://arxiv.org/pdf/0903.4726.pdf

# Test Cases

In [3]:
#Test Case 1 - Given 
lst = [ 6, 2, 0, 7, 9, 3, 1, 8, 5, 4]
RQQ(lst,5,3,9)

Wavelet Tree is: 
Level  0 :  1001100110  
Level  1 :  00101 00110  
Level  2 :  100 01 010 10  
Level  3 :  01 X X X 10 X X X  
-----------------------
Output of Range Quantile Query: 
Level  0 :  (5, 3, 9)
Level  1 :  (2, 2, 5)
Level  2 :  (2, 2, 3)
Level  3 :  (1, 1, 1)
Answer:  7


In [4]:
#Test Case 2 - Element in Left subtree 
lst = [6, 2, 0, 7, 9, 3, 1, 8, 5, 4]
RQQ(lst,5,1,8)

Wavelet Tree is: 
Level  0 :  1001100110  
Level  1 :  00101 00110  
Level  2 :  100 01 010 10  
Level  3 :  01 X X X 10 X X X  
-----------------------
Output of Range Quantile Query: 
Level  0 :  (5, 1, 8)
Level  1 :  (1, 1, 4)
Level  2 :  (1, 1, 2)
Level  3 :  (1, 1, 1)
Answer:  6


In [5]:
#Test Case 3 - Passing the entire list in the ascending order
lst = list(range(1,10))
RQQ(lst,4,1,9)

Wavelet Tree is: 
Level  0 :  000001111  
Level  1 :  00011 0011  
Level  2 :  001 01 01 01  
Level  3 :  01 X X X X X X X  
-----------------------
Output of Range Quantile Query: 
Level  0 :  (4, 1, 9)
Level  1 :  (4, 1, 5)
Level  2 :  (1, 1, 2)
Level  3 :  (1, 1, 1)
Answer:  4


In [6]:
#Test Case 4 - Passing the entire list in the descending order
lst = list(range(10,1,-1))
RQQ(lst,4,1,9)

Wavelet Tree is: 
Level  0 :  111100000  
Level  1 :  11000 1100  
Level  2 :  100 10 10 10  
Level  3 :  10 X X X X X X X  
-----------------------
Output of Range Quantile Query: 
Level  0 :  (4, 1, 9)
Level  1 :  (4, 1, 5)
Level  2 :  (1, 1, 2)
Level  3 :  (1, 1, 1)
Answer:  5


In [7]:
#Test Case 5 - With only negative numbers in the list
lst = list(range(-10,-1,1))
RQQ(lst,7,1,9)

Wavelet Tree is: 
Level  0 :  000001111  
Level  1 :  00011 0011  
Level  2 :  001 01 01 01  
Level  3 :  01 X X X X X X X  
-----------------------
Output of Range Quantile Query: 
Level  0 :  (7, 1, 9)
Level  1 :  (2, 1, 4)
Level  2 :  (2, 1, 2)
Level  3 :  (1, 1, 1)
Answer:  -4


In [8]:
#Test Case 6 - Combination of positive and negative numbers in the list
lst = [0,1,5,3,-1,-6,9,-10,4,-9,-12,6]
RQQ(lst,5,4,11)

Wavelet Tree is: 
Level  0 :  111110101001  
Level  1 :  1000 00100101  
Level  2 :  110 X 00101 010  
Level  3 :  X 01 010 01 01 X  
Level  4 :  X X 10 X X X X X  
-----------------------
Output of Range Quantile Query: 
Level  0 :  (5, 4, 11)
Level  1 :  (1, 4, 7)
Level  2 :  (1, 3, 5)
Level  3 :  (1, 3, 3)
Level  4 :  (1, 2, 2)
Level  5 :  (1, 1, 1)
Answer:  -1


In [9]:
#Test Case 7 - Invalid parameter 
lst = [6, 2, 0, 7, 9, 3, 1, 8, 5, 4]
RQQ(lst,8,1,5)

Wavelet Tree is: 
Level  0 :  1001100110  
Level  1 :  00101 00110  
Level  2 :  100 01 010 10  
Level  3 :  01 X X X 10 X X X  
-----------------------
Enter valid parameters
